In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
print(f'Running on TPU {tpu.master()}')
# Running on TPU grpc://10.0.0.2:8470

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


2024-03-27 09:28:08.925285: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-03-27 09:28:08.925395: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-03-27 09:28:08.925501: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-03-27 09:28:08.925585: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-03-27 09:28:08.925660: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-03-27 09:28:08.925874: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this stream.
2024-03-27 09:28:08.925964: E external/local_xla/xla/stream_executor/stream_executor_internal.h:177] SetPriority unimplemented for this 

INFO:tensorflow:Finished initializing TPU system.
Running on TPU 


In [ ]:
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
from tensorflow.keras import layers, applications, optimizers, losses

In [ ]:
GCS_PATH = KaggleDatasets().get_gcs_path('tpu-getting-started') + '/tfrecords-jpeg-512x512'
print(GCS_PATH)

/kaggle/input/tpu-getting-started/tfrecords-jpeg-512x512


get_gcs_path is not required on TPU VMs which can directly use Kaggle datasets, using path: /kaggle/input/tpu-getting-started


In [ ]:
filenames = tf.io.gfile.glob(GCS_PATH + "/train/*.tfrec")

In [ ]:
dataset = tf.data.TFRecordDataset(filenames)

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.reshape(image, [512, 512, 3])
    return image

def read_labeled_tfrecord(example):
    LABELED_TRFEC = {
        "image": tf.io.FixedLenFeature([], tf.string),
        "class": tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, LABELED_TRFEC)
    image = decode_image(example['image'])
    label = tf.cast(example["class"], tf.int32)
    return image, label

dataset = dataset.map(read_labeled_tfrecord)

In [ ]:
def data_augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.rot90(image , k = 1)
    image = tf.image.random_brightness(image, max_delta=.2)
    return image, label

dataset = dataset.map(data_augment)

In [ ]:
dataset = dataset.repeat()
dataset = dataset.shuffle(2048)
dataset = dataset.batch(128)
dataset = dataset.cache()
dataset = dataset.prefetch(128*3)

In [ ]:
def create_model():
    base_model = applications.EfficientNetB6(weights="imagenet",
                                             include_top=False,
                                             input_shape=(512, 512, 3))

    model = tf.keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(104, activation="softmax")
    ])

    return model

# Define the distribution strategy
strategy = tf.distribute.experimental.TPUStrategy(tpu)

with strategy.scope():
    # Create the model within the scope of the strategy
    model = create_model()

    # Compile the model within the same strategy scope
    model.compile(optimizer=optimizers.Adam(),
                  loss="sparse_categorical_crossentropy")

# Now, train the model with the same distribution strategy
model.fit(dataset,
          steps_per_epoch=100,
          epochs=20)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

Epoch 1/20


2024-03-27 10:02:20.594672: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node StatefulPartitionedCall.


100/100 ━━━━━━━━━━━━━━━━━━━━ 226s 2s/step - loss: 0.0000e+00
Epoch 2/20


/usr/local/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0000e+00
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0000e+00
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0000e+00
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0000e+00
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0000e+00
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0000e+00
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0000e+00
Epoch 9/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0000e+00
Epoch 10/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0000e+00
Epoch 11/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0000e+00
Epoch 12/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0000e+00
Epoch 13/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0000e+00
Epoch 14/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0000e+00
Epoch 15/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.0000e+00
Epoch 16/20

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def predict_flower_category(image_path, model):
    # Load and preprocess the image
    image = tf.io.read_file(image_path)
    image = decode_image(image)
    image = tf.expand_dims(image, axis=0)  # Add batch dimension

    # Predict the category
    predicted_logits = model.predict(image)
    predicted_class = np.argmax(predicted_logits)

    return predicted_class

def show_similar_flowers(image_path, model, dataset, num_images=5):
    # Predict the category of the input image
    predicted_class = predict_flower_category(image_path, model)

    # Filter the dataset to get images of the predicted category
    images_of_predicted_class = []
    for image, label in dataset:
        if label == predicted_class:
            images_of_predicted_class.append(image)
        if len(images_of_predicted_class) == num_images:
            break

    # Show the images
    plt.figure(figsize=(15, 6))
    for i, image in enumerate(images_of_predicted_class):
        plt.subplot(1, num_images, i+1)
        plt.imshow(image.numpy())
        plt.axis('off')
        plt.title(f'Flower {i+1}')
    plt.show()

# Example usage
# image_path = 'path/to/your/image.jpg'  # Replace with the path to your image
# show_similar_flowers(image_path, model, dataset)
